# GPU Programming with CuPy

*Disclaimer: Part of this notebook has been inspired by the [CuPy documentation](https://docs-cupy.chainer.org/en/stable/)*, which is also a great resource for a more detailed description of the CuPy API.

In [ ]:
import timeit
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

Santis has a node architecture where each node has four Hopper GPUs and four Grace CPUs (see [Grace Hopper Module](https://docs.cscs.ch/alps/hardware/#nvidia-gh200-gpu-nodes)). It consists of around 430 Grace-Hopper nodes connected with a network (see [Santis](https://docs.cscs.ch/clusters/santis/#cluster-specification)). Since we are currently working on one of these nodes, we have access to four GPUs, but only need 1 for the course.

Let's take a look if that's really the case. We can use the `nvidia-smi` command to inspect whether there is a GPU available.

In [ ]:
%%bash
nvidia-smi

So we can see that we have four GH200 with IDs 0-3. We can also see that we have ~98 GB of memory on each GPU. We can further see the operating temperature and the power consumption in Watt. The second box lists the programs currently running on the GPU. If at all, there might be only one entry which is our current notebook process since we have already run `import cupy` above.

We can manage all four GPUs using the ID to identify them and specifying the appropriate slurm commands. By default cupy will select ID:0 and stick with it, which is exactly what we want for our experiments.

## Hybrid Node Architecture

Each node of Santis has four Grace Hopper modules shown below. It is interesting to consider the different node-internal connections and put them into perspective. The following illustration shows the bandwidth of the connections between them and their respective memories, and the size of the memories.

![hybrid node architecture](img/cpu_gpu_bw.png)

In terms of FLOPS the Hopper GPU (67 FLOPS for FP64) outperforms the Grace CPU (<=2.5 FLOPS for FP64) by a factor of almost 30 (see [Slide 11](https://www.cscs.ch/fileadmin/user_upload/contents_publications/tutorials/Grace-Hopper_getting_ready.pdf)).

It is important to keep in mind that only the CPU is actually connected to the internet. And unless specific frameworks like [GPUDirect Storage](https://developer.nvidia.com/blog/gpudirect-storage/) are used, the GPU can't access the system storage directly.

## Memory Managment and Data Transfer

A fundamental difficulty of GPU programming, is that the GPU does not directly share the memory with the CPU. This means that we have to actively manage where data is placed as well as data movement. For example, if we want to print data on the GPU to the screen, the data has to be copied from the GPU memory to the CPU memory and only then we can display it on the screen. Inversely, if we have loaded a data set from disk, it will reside in the memory of the CPU. We have to explicitly copy the data to the GPU memory before we can work on it on the GPU.

While GPUs are much faster than CPUs (higher peak FLOP/s and higher memory bandwidth), one of the main difficulties of GPU programming is making sure that the benefit of this faster processing speed is not offset by the overhead of moving the data from the CPU to the GPU and the results back.

Recent improvements in frameworks like NVLink allow for much faster CPU-GPU communication. In the bonus exercises we will investigate the complex world of [Data Movement in Tightly Coupled Heterogeneous Systems](https://arxiv.org/html/2408.11556v1) further.

Luckily, CuPy - in contrast to other GPU programming models such as CUDA or OpenACC - handles some of the memory management complexity for us. Let's start out with a simple example.

In [ ]:
# create a data array in CPU memory
x_cpu = np.array([1, 2, 3])

# create a data array in GPU memory
x_gpu = cp.array([1, 2, 3])

# show values
print(x_cpu)
print(x_gpu)

Didn't we just say that we cannot print data values which reside in GPU memory? While this is true in principle, CuPy handles the copy from GPU to CPU and the print for us. But we can see that there are limits to this "illusion" when we try to assign the values of one of these arrays to the other.

In [ ]:
x_cpu[:] = x_gpu[:]  # Error

While this would be a perfectly legal operation for two regular NumPy arrays, assigning values of an array in GPU memory to an array in CPU memory cannot be done. This is very similar to the distributed memory situation where we had to use the MPI library to send messages from one rank to another in order to transfer data, because the different ranks did not share the same memory space.

The `.device` attribute contains the GPU device ID or the string cpu respectively.

In [ ]:
print(x_gpu.device)  # OK
print(x_cpu.device)  # Error

In order to move data from the CPU memory (host) to the GPU memory (device) we have to use the `cupy.asarray()` function. In order to move data from the GPU memory (device) back to the CPU memory (host) we can use the `get()` method of CuPy arrays.

In [ ]:
del x_cpu, x_gpu

In [ ]:
# generate an array on the host (CPU)
cp.cuda.profiler.start()
x_cpu = np.linspace(0., 10., 100)

# copy the data to the device (GPU)
x_gpu = cp.asarray(x_cpu)

# do some computation on the GPU
y_gpu = cp.sin(x_gpu)

# copy the result back to the host (CPU)
y_cpu = y_gpu.get()

cp.cuda.profiler.stop()

# show the result
plt.plot(x_cpu, y_cpu);

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>1.</b> Define two matrices A and B of size 4096 x 4096 and initialize them with random numbers. Copy them to the GPU. Execute matrix multiplication on the GPU (you can used the <tt>cp.dot()</tt> method) and store the result in C. Copy the result back to the CPU. Validate your result using <tt>np.allclose()</tt> against a CPU reference calculation.<br>
</div>

In [ ]:
# allocate and initialize matrices on CPU
a_np = None#TODO
b_np = None#TODO

# transfer matrices to GPU
a_cp = None#TODO
b_cp = None#TODO

# perform matrix-matrix multiplication on GPU
c_cp = None#TODO

# move the result back to CPU
c_np = None#TODO

# validate
c_ref_np = None#TODO
assert np.allclose(c_np, c_ref_np)

# Synchronization

CuPy adopts an asynchronous model for GPU execution. This basically means that the CPU simply tells the GPU what work it would like to be done and the CuPy call returns immediately while the GPU is doing the work in the background. (For GPU enthusiasts, this means that a kernel is being launched in a stream.)

Similar to the `!$omp barrier` directive which synchronizes a team of threads and the `MPI.COMM_WORLD.Barrier()` which synchronizes a set of processes, CuPy has the `cp.cuda.Device().synchronize()` method which synchronizes CPU and GPU execution. Basically, `cp.cuda.Device().synchronize()` waits until both the GPU and the CPU have arrived at this statement in the program execution.

In [ ]:
num_elements = 256 * 1024 * 1024
x_gpu = cp.random.rand(num_elements)

tic = timeit.default_timer()
y_gpu = cp.sin(x_gpu)
toc = timeit.default_timer()
elapsed_time_work = toc - tic

tic = timeit.default_timer()
cp.cuda.Device().synchronize()
toc = timeit.default_timer()
elapsed_time_sync = toc - tic

print(f"Work time = {1e6 * elapsed_time_work:.6f} µs")
print(f"Sync time = {1e6 * elapsed_time_sync:.6f} µs")

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>2.</b> Change the number of elements to 1 and then gradually increase back to the original value (256 x 1024 x 1024). Does the work time or sync time increase?<br>
<b>3.</b> Add another synchronization just before the first tic to make sure that the CPU and GPU are in sync. Then add another synchronization just before the first toc to make sure that the GPU has finished doing all of its work. Now run again and check how the timers behave when you change the number of elements in the data array.<br>
</div>

It is probably best to define a method for timing that we can use in the future which already includes the syncronization in order to make sure we don't forget it and our timings are off. The above example illustrates how wrong our interpretation of the performance can be if we don't take care to synchronize.

In [ ]:
def get_time(sync=True):
    if sync:
        cp.cuda.Device().synchronize()
    return timeit.default_timer()

In [ ]:
# TODO: Loop over elements and print times

In [ ]:
# TODO: Check where syncs have what effects

## Performance and Speedup

One might wonder why one would go to all the trouble of doing all of this work just to compute a `np.sin()` which we could have done much simpler without a GPU. The main reason why we want to use a GPU and are willing to go to the trouble of dealing with host / device memory management is because GPUs often allow us to do things much faster.

If you've ever run into the situation where your Python analysis script or your Fortan code simply started taking much too long for your laptop to execute, this is where accelerators such as GPUs come in.

In [ ]:
num_elements = 64 * 1024 * 1024
x_cpu = np.random.rand(num_elements)

tic = get_time()
y_cpu = np.arctan(x_cpu)
elapsed_time_cpu = get_time() - tic

x_gpu = cp.asarray(x_cpu)

tic = get_time()
y_gpu = cp.arctan(x_gpu)
elapsed_time_gpu = get_time() - tic

speedup = elapsed_time_cpu / elapsed_time_gpu
print(f"Time CPU = {elapsed_time_cpu:.6f} s")   
print(f"Time GPU = {elapsed_time_gpu:.6f} s ({speedup:.3f}x speedup)")

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>4.</b> While the speedup reported above is very impressive, it neglects the time it takes to copy data to the GPU and the result back from the GPU. Insert code to copy the result back. Insert timers around the copy to the GPU and the copy back from the GPU. Make sure you synchronize before a time measurement. Add a print for the speedup relative to the total time including data movement to/from the GPU.<br>
<b>5.</b> The default is an array with 64 x 1024 x 1024 floating point values (each 8 Bytes). Compute how many MB we are transfering to the GPU and back. Compute the bandwidth in GB/s for the transfers to/from the GPU.<br>
<b>6.</b> Try out different number of elements in the array and inspect how speedup changes. What is the minimal array size for a speedup when comparing only the work and when comparing total time including data transfers? What is the maximum speedup you can achieve?<br>
<b>7. (Bonus)</b> Try it out with a simple copy, a simple stencil (e.g. <code>0.5 * (a[i+1] + a[i] + a[i-1])</code> or a matrix-matrix multiplication. How does the maximum speedup change?
</div>

In [ ]:
# TODO - speedup including transfers

**TODO**

comuptation of bandwidth

In [ ]:
# TODO - Vary the size

## Array Programming and Vectorization (and its limits)

In order to get good performance with CuPy (and NumPy) it is key to use array programming in favor of explicit loop as much as possible. Let's compare the runtime of a vector addition (daxpy) using both array programming and an explicit loop.

In [ ]:
size = 64 * 1024
for xp, backend in zip([cp, np], ["cupy", "numpy"]):

    x = xp.random.rand(size)
    y = xp.random.rand(size)
    z = xp.empty_like(x)
    a = xp.pi
  
    tic = get_time()
    z[...] = a * x + y
    elapsed_time_array = get_time() - tic
    
    tic = get_time()
    for i in range(size):
        z[i] = a * x[i] + y[i]
    elapsed_time_loop = get_time() - tic
    
    print(
        f"{backend} \t loop={elapsed_time_loop:.5f}s \t "
        f"array={elapsed_time_array:.5f}s \t speedup={elapsed_time_loop / elapsed_time_array:.3f}"
    )

One can see that it is essential to use array programming notation. Both backends have a larger hit in terms of performance when calculating explicit loops over our data arrays.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>8.</b> Try writing the following motifs using array programming:
<ul>
<li>Simple stencil<br>
    $\;\;\phi_i = \left( \phi_{i-1} - 2 \, \phi_i + \phi_{i+1} \right) / \Delta x$<br></li>
<li>Upstream scheme<br>
    $\;\;\phi_i = \begin{cases} u_i \left( \phi_i - \phi_{i-1} \right) / \Delta x \;\;\; u_i \ge 0\\ u_i \left( \phi_{i+1} - \phi_i \right) / \Delta x \;\;\; u_i \lt 0 \end{cases}$<br></li>
<li>Maximum in neighborhood with configurable $n=3$ (assume cyclic boundary conditions)<br>
    $\;\;\phi_i = \max\limits_{j=-n,n} \phi_{i + j}$<br></li>
<li>Fixed-point iteration until converged with $\alpha = 0.01$ and $\epsilon = 1e-4$<br>
    $\;\;\mathrm{while} \,\, \left| \phi_{n+1} - \phi_{n} \right| > \epsilon:$<br>
    $\;\;\;\;\;\phi_{n+1} = (1 - \alpha) \, \phi_n + \alpha \, \phi_n^3$<br></li>
</ul>
</div>

In [ ]:
# TODO - vectorization and array programming

While often it is possible to write the code using array programming (in vector form), it is not always easy. Some algorithms might actually be very hard, cumbersome or impossible to write using array programming. Typical examples are algorithms with data dependencies, algorithms with variable work depending on the position in the array, or algorithms which work on only a subset of the data array.

CuPy as well as NumPy provide some methods to extend what is possible with pure array programming syntax via the `numpy.vectorize` or the `cupy.ElementwiseKernel` and `cupy.ReductionKernel`, or even by being able to plug-in raw CUDA code (see below) by using the `cupy.RawKernel` method. Typically these methods do what we call *break the abstraction*, namely the code starts to diverge between a NumPy and a CuPy implemention, is hardware specific and no longer portable, or requires the knowledge of another programming language and is no longer part of the Python language (e.g. `cupy.RawKernel`).

## Writing CPU/GPU Agnostic Code

CuPy can serve as a drop-in replacement of NumPy in many aspects. But your code will most probably not always be running on a system where there is a GPU present. Or you would like to be able to easily switch back and forth between using NumPy and CuPy in order to test, develop and/or debug your program. A simple method of writing a CPU/GPU agnostic code is the following.

In [ ]:
import numpy as np
try:
    import cupy as xp
except ImportError:
    xp = np

It is important to note that the code is slightly different from "normal" NumPy code you would write, in that you have to use the `get()` method whenever you would do a transfer from GPU memory to CPU memory (for example for plotting or for writing data to disk). Similarly, when reading data from disk you will initially have a NumPy array (irrespective of the backend you are using for `xp`) and using the method `xp.asarray()` will make sure that the data is transferred to the GPU in the case of using CuPy.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>9.</b> Try forcing the above code that computes a sin on a linspace to run using NumPy and CuPy and make sure that it runs with both.
</div>

In [ ]:
import numpy as np
try:
    import cupy as xp
except ImportError:
    xp = np
    
def get_array(z):
    # TOOD
    pass

# this is a CPU/GPU agnostic program
x = None # TODO
y = None # TODO
plt.plot(get_array(x), get_array(y))

## Low-level GPU Programming

Computations on the GPU with CuPy are extremently convenient and powerful. But it is by far not the only or most common way that GPUs are being used (and as we've seen above also has its limitations). But why can we not simply use Fortran or C/C++ to program GPUs? The reason is the same as for parallel programing with OpenMP. Most programming languages have not been designed for parallel programming and additional information from the programmer is required because the compilers are (often) not able to automatically parallelize code written in Fortran or C/C++. Furthermore, the programmer also has to explicitly manage data transfers between the CPU and the GPU, in case performance is of any importance.

#### OpenACC Directives

One popular low-level approach to GPU programming are [OpenACC](https://www.openacc.org) compiler directives which can be used in a similar fashion to OpenMP to port an existing Fortran or C/C++ code to GPU (see for example [this tutorial](https://github.com/C2SM-RCM/OpenACC_Training)). Here's an example of OpenACC when used with Fortran.

```Fortran
  ! driving routine for the physical parameterizations
  SUBROUTINE physics()
  IMPLICIT NONE

  INTEGER :: i,j    ! loop indices

  !$acc data present(t,qc,qv) 
  !$acc parallel
  !$acc loop gang
  DO j = 1,ny
  
    !$acc loop vector
    DO i = 1,nx
      DO k = 1, nlev
        v(i,j,k) = qv(i,j,k) + cs1*EXP(cs2*( t(i,j,k)-t0 )/( t(i,j,k)-cs3) )
        qc(i,j,k) = cs4*qv(i,j,k)
      END DO
      DO k = 2, nlev
        qv(i, j, k) = qv(i, j, k-1) + cm1*(t(i, j, k)-cm2)**cm3
        t(i, j, k)  = t(i, j, k)*( 1.0D0 - cm4*qc(i, j, k)+qv(i, j, k) )
      END DO
    END DO

  END DO
  !$acc end parallel
  !$acc end data
  
  END SUBROUTINE physics
```

You can see that there are `!$acc data` statements which are used for data movement / management. There is a `$!acc parallel` and a `!$acc loop` statement which is the analogue to the `!$omp parallel do` statement. 

#### CUDA

CUDA (Compute Unified Device Architecture) is an extension of the C/C++ programming language and can be used for programming GPUs from NVIDIA. Take a look at [this tutorial](https://www.nvidia.com/content/GTC-2010/pdfs/2131_GTC2010.pdf) if you would like to learn more. An example of a simple CUDA program can be seen below. The `main()` is essentially standard C/C++ except for the decoration of the invocation of the kernel function with `<<<numBlocks, blockSize>>>` and the `__global__` keyword in front of the declaration of the kernel function indicating that this is a function that should run on the GPU. Multiple threads will be running the `add()` function in parallel and in the example below an operation such as `y[i] = x[i] + 0.5 * ( y[i+1] + y[i-1] )` would be a race condition and give undefined results.

```C++
#include <iostream>
#include <math.h>

// kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // allocate GPU memory
  cudaMalloc(&x, N*sizeof(float));
  cudaMalloc(&y, N*sizeof(float));

  // initialize x and y arrays (to zero)
  cudaMemset(x, 0, N*sizeof(float));
  cudaMemset(y, 0, N*sizeof(float));

  // run kernel
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);

  // free memory
  cudaFree(x);
  cudaFree(y);
  
  return 0;
}
```

#### Accessing Low-Level Programming from CuPy

CuPy allows access to low-level programming with CUDA via the `cupy.RawKernel` method. In order to use this, one has to know the CUDA programming language and have a working knowledge of how GPUs actually work. Below is an example of how one can compile a custom CUDA kernel and launch it from Python.

In [ ]:
# define a kernel that adds the column vector y (1d) to all columns of x (2d)
add_and_broadcast_kernel = cp.RawKernel(
    r'''
    extern "C" __global__
    void add_and_broadcast_kernel(
        const float* x, const float* y, float* z,
        const int xdim, const int ydim)
    {
        int idx_x = blockIdx.x * blockDim.x + threadIdx.x;
        int idx_y = blockIdx.y * blockDim.y + threadIdx.y;
        z[idx_y * xdim + idx_x] = x[idx_y * xdim + idx_x] + y[idx_x];
    }
    ''',
    'add_and_broadcast_kernel'
)

# 
def launch_add_and_broadcast_kernel(x, y):
    block_size = (32, 32)
    grid_size = (x.shape[1] // block_size[1], x.shape[0] // block_size[0])

    z = cp.empty(x.shape, x.dtype)
    xdim = x.strides[0] // x.strides[1]
    ydim = z.strides[0] // z.strides[1]

    add_and_broadcast_kernel(grid_size, block_size, (x, y, z, xdim, ydim))
    return z

# allocate storages on GPU
xdim = 4096
ydim = 1024
x = cp.arange(xdim * ydim, dtype=cp.float32).reshape((xdim, ydim))
y = cp.arange(ydim, dtype=cp.float32)

# run the custom kernel
z = launch_add_and_broadcast_kernel(x, y)

# verify the result
print(cp.all(x + y == z))

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>10.</b> The file  <tt>stencil2d-cupy.py</tt> is a copy of the file we used in day 1. Your goal here is to port the Python program to GPUs using CuPy and what you have learned in this notebook.<br>
<b>11.</b> Run the original <tt>stencil2d-original.py</tt> and your new GPU-enabled version for 16x16, 32x32, 64x64, 96x96 and 128x128 horizontal grid points and make a plot of speedup versus grid size.<br>
</div>

In order to run the codes starting here, you need to stop the kernel running here and restart it (so we don't use the GPU anymore)

In [ ]:
%%bash
# Validation of the cupy code
./validation.sh

In [ ]:
from IPython.display import Image, display
display(Image(filename='out_field.png'))

**Reference image for 16x16x80 for 20 steps**

![img/out_field_ref.png](img/out_field_ref.png)


In [ ]:
%%bash
# Validation of the cupy code
./validation.sh

In [ ]:
%%bash
# TODO Exercise 12
srun -n 1 python stencil2d-agnostic.py --nx=16 --ny=16 --nz=80 --num_iter=5

In [ ]:
%%bash
# TODO Exercise 11
srun -n 1 python stencil2d-original.py --nx=16 --ny=16 --nz=80 --num_iter=5

<div class="alert alert-block alert-success">
<b>Now it's your turn...</b><br>
<b> Bonus 12 </b> Try to make it so, that the program will evaluate with NumPy in case no CuPy is present in <tt>stencil2d-agnostic.py</tt><br>
<b>Bonus 13</b> Combine your work for the MPI parallel version from the day3 work with the GPU-enabled version we just created. Test your program by validating the output against the serial CPU reference code and by reading the <tt>out_field.npy</tt> data from each version. If you've validated your parallel version, talk to us to give it a spin on multiple nodes on Piz Daint. Congrats, you have just written a parallel stencil program that could potentially run on all of Piz Daint using it's resources efficiently!<br>
<b>Bonus 14</b> Try out different memory management systems for the data transfer between CPU and GPU memory. You can use this paper as an overview of GH200's memory handling: https://arxiv.org/html/2408.11556v1</tt><br>
<b>Bonus 15</b> TODO: Compiling Fortran With OpenACC is Currently not Supported on ALPS! 
If you are already familiar with GPU computing, take a stab at porting the <tt>stencil2d.F90</tt> using OpenACC or the <tt>stencil2d.cpp</tt> using CUDA.<br><br>
</div>

In [ ]:
%%bash
# TODO Exercise 12
srun -n 1 python stencil2d-agnostic.py --nx=16 --ny=16 --nz=80 --num_iter=5

In [ ]:
%%bash
# TODO Exercise 13
srun -n 9 python stencil2d-parallel.py --nx=16 --ny=16 --nz=80 --num_iter=5